# Script to Identify Client issues

The script is split into 2 parts to readabilty and make the code manageable
  1. To Load the required nuget and setup DI, initalize some utility code
  2. The main scripts
How to run this?
1. Execute/ Run Cell-1, this loads the required nuget packages and setups up the service provider and scope

In [33]:
#r "nuget: MortgageClient.Api.Client"
#r "nuget: Microsoft.Extensions.DependencyInjection, 6.0.0"
#r "nuget: Microsoft.Extensions.Configuration, 6.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 6.0.0"
#r "nuget: DecisionServices.Core.Extensions.Authentication, 8.12.0"
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Data.Analysis"

using System;
using Microsoft.Extensions.DependencyInjection;
using MortgageClient.Api.Client;
using MortgageClient.Api.Client.Models;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.Configuration.Json;
using DecisionServices.Core;
using DecisionServices.Core.Authentication;
using System.Net.Http;
using Newtonsoft.Json;
using Microsoft.DotNet.Interactive;
using MCClient = MortgageClient.Api.Models.ReadClient;
using Microsoft.Data.Analysis;
using System.Collections.Generic;
using System.Text;

#region Data Frame Pretty Print
    // from https://swharden.com/blog/2022-05-01-dotnet-dataframe/
    public static void PrettyPrint(this DataFrame df) => Console.WriteLine(PrettyText(df));
    public static string PrettyText(this DataFrame df) => ToStringArray2D(df).ToFormattedText();

    private static string[,] ToStringArray2D(this DataFrame df)
    {
        string[,] strings = new string[df.Rows.Count + 1, df.Columns.Count];

        for (int i = 0; i < df.Columns.Count; i++)
            strings[0, i] = df.Columns[i].Name;

        for (int i = 0; i < df.Rows.Count; i++)
            for (int j = 0; j < df.Columns.Count; j++)
                strings[i + 1, j] = df[i, j]?.ToString() ?? string.Empty;

        return strings;
    }

    private static int[] GetMaxLengthsByColumn(this string[,] strings)
    {
        int[] maxLengthsByColumn = new int[strings.GetLength(1)];

        for (int y = 0; y < strings.GetLength(0); y++)
            for (int x = 0; x < strings.GetLength(1); x++)
                maxLengthsByColumn[x] = Math.Max(maxLengthsByColumn[x], strings[y, x].Length);

        return maxLengthsByColumn;
    }

    private static string ToFormattedText(this string[,] strings)
    {
        StringBuilder sb = new();
        int[] maxLengthsByColumn = GetMaxLengthsByColumn(strings);

        for (int y = 0; y < strings.GetLength(0); y++)
        {
            for (int x = 0; x < strings.GetLength(1); x++)
            {
                sb.Append(strings[y, x].PadRight(maxLengthsByColumn[x] + 2));
            }
            sb.AppendLine();
        }

        return sb.ToString();
    }

#endregion

#region Constants
  const string Name ="Name";
  const string ClientUID = "Client UID";
  const string GCID ="GCID";
  const string Active ="Active";
#endregion

#region Open Amp Models
  public class Client
  {
    public int ApplicationNumber { get; set; }
    public int BorrowerNumber { get; set; }
    public string LastName { get; set; }
    public string FirstName { get; set; }
    public string MiddleName { get; set; }
    public string GCUID { get; set; }
  }
  public class Clients
  {
    public List<Client> Client { get; set; }
  }

  public class AmpClient
  {
    public Clients Clients {get; set;}
  }

  public class LoanJacketDetail
  {
    public string JacketNumber { get; set; }
    public string ApplicationNumber { get; set; }
  }
#endregion

#region OpenAmp
  interface IOpenAmpService {
    Task<IEnumerable<Client>> GetClients(string loanNumber);
  }
  class OpenAmpService: IOpenAmpService 
  {
    private readonly HttpClient _httpClient;
    public OpenAmpService(HttpClient httpClient)
    {
      _httpClient = httpClient;
    }
    public async Task<IEnumerable<Client>> GetClients(string loanNumber)
    {
      var applicationNumber = await GetApplicationNumber(loanNumber);
      var getAllClientsUri = $"Proxy/Client?ApplicationNumber={applicationNumber}";
      var ampClients =await GetAsync<AmpClient>(getAllClientsUri, _httpClient);
      if (!ampClients?.Clients?.Client?.Any() ?? true)
      {
        return null;
      }
      return ampClients?.Clients?.Client;
    }

    private async Task<string> GetApplicationNumber(string loanNumber)
    {
      var loanDetail = (await GetAsync<IEnumerable<LoanJacketDetail>>($"Application/Jacket/{loanNumber}", _httpClient))
                  .FirstOrDefault();
      if (loanDetail == null)
      return null;

      return loanDetail?.ApplicationNumber;
    }
  }
#endregion

#region Http GET call
  public static async Task<T> GetAsync<T>(string url, HttpClient httpClient)
  {
    var response = await httpClient.GetAsync(url);
    response.EnsureSuccessStatusCode();
    var responseBody = await response.Content.ReadAsStringAsync();
    return JsonConvert.DeserializeObject<T>(responseBody)!;
  }
#endregion

var services = new ServiceCollection();
services.SetupClients();

static void SetupClients(this ServiceCollection services)
{
  var config = new ConfigurationBuilder()
  .SetBasePath(Environment.CurrentDirectory)
              .AddJsonFile("appsettings.json").Build();
  var openAmpConfig = new AuthenticatedServiceConfig()
  {
    Audience = config.GetValue<string>("OpenAmp:Audience"),
    BaseUrl = config.GetValue<string>("OpenAmp:BaseUrl"),
    ClientId = config.GetValue<string>("Auth0:ClientId"),
    ClientSecret = config.GetValue<string>("Auth0:ClientSecret"),
    TokenEndpoint = config.GetValue<string>("Auth0:TokenEndpoint")
  };
  services.AddMortgageClientSdk(new MortgageClientApiOptions()
    {
      Audience = config.GetValue<string>("MortgageClient:Audience"),
      BaseUrl = config.GetValue<string>("MortgageClient:BaseUrl"),
      ClientId = config.GetValue<string>("Auth0:ClientId"),
      ClientSecret = config.GetValue<string>("Auth0:ClientSecret"),
      TokenEndpoint = config.GetValue<string>("Auth0:TokenEndpoint")
    }).AddNewtonsoftSerialization();
  services.AddAuthenticatedService<IOpenAmpService, OpenAmpService>(
                    openAmpConfig,
                    client =>
                    {
                        client.DefaultRequestHeaders.Add("xampuserid", "mcimport");
                        client.DefaultRequestHeaders.Add("vnd.rockfin.appid", "210418");
                    });
}

var serviceProvider = services.BuildServiceProvider();
var scope = serviceProvider.CreateScope();
var mcServiceClient = scope.ServiceProvider.GetRequiredService<IClientService>();
var ampServiceClient = scope.ServiceProvider.GetRequiredService<IOpenAmpService>();

static DataFrame ToDataFrame(this Dictionary<string, List<string>> data)
{
  DataFrameColumn[] columns = new DataFrameColumn[data.Keys.Count];
  data.Select((item, index) => new {item, index})
  .ToList()
  .ForEach(entry => {
    columns[entry.index] = new StringDataFrameColumn(entry.item.Key, entry.item.Value.ToArray());
  });
  DataFrame df = new(columns);
  return df;
}

static void Display(this IEnumerable<MCClient> mcClients)
{
  var dfDictionary = new Dictionary<string, List<string>>()
  {
    {Name, new List<string>()},
    {ClientUID, new List<string>()},
    {GCID, new List<string>()},
    {Active, new List<string>()}
  };
  mcClients.ToList().ForEach(c => {
    var active = c.IsActive? '✓': '❌';
    dfDictionary[Name].Add($"{c.PersonalInformation?.FirstName} {c.PersonalInformation?.LastName}");
    dfDictionary[ClientUID].Add(c.RocketLogicClientId);
    dfDictionary[GCID].Add(c.GlobalClientId);
    dfDictionary[Active].Add(active.ToString());
  });
  Console.WriteLine("MC Clients");
  dfDictionary.ToDataFrame().PrettyPrint();
}

static void Display(this IEnumerable<Client> ampClients)
{
  var dfDictionary = new Dictionary<string, List<string>>()
  {
    {Name, new List<string>()},
    {GCID, new List<string>()},
  };
  ampClients.ToList().ForEach(c => {
    dfDictionary[Name].Add($"{c.FirstName} {c.LastName}");
    dfDictionary[GCID].Add(c.GCUID);
  });
  Console.WriteLine("Amp Clients");
  dfDictionary.ToDataFrame().PrettyPrint();
}

static void VerifyClientsMatch(List<MCClient> mcClients, List<Client> ampClients)
{
  Print("Checking if clients numbers match..");
  var mcCount = mcClients?.Count ?? 0;
  var ampCount = ampClients?.Count ?? 0;
  if (mcCount != ampCount)
  {
    Print($"❌ Client Counts don't Match  MC: {mcCount}, AMP: {ampCount}");
  }
  Print($"✓ MC and AMP Client Counts Match");
  return;
}

static void CheckGCIDMisMatch(List<MCClient> mcClients, List<Client> ampClients)
{
  Print("Checking if clients numbers match..");
  mcClients.ForEach(c => {
    
  });
}

static void CheckDuplicateGCID(List<MCClient> mcClients, List<Client> ampClients)
{
  Console.WriteLine("Checking for Duplicate GCID");
}


static void Print(string message)
{
  Console.WriteLine(message);
}
static void PrintNewLine(string message)
{
  Print(message);
  Console.WriteLine();
}

Installed Packages DecisionServices.Core.Extensions.Authentication, 8.12.0 Microsoft.Data.Analysis, 0.20.1 Microsoft.Extensions.Configuration, 6.0.0 Microsoft.Extensions.Configuration.Json, 6.0.0 Microsoft.Extensions.DependencyInjection, 6.0.0 MortgageClient.Api.Client, 3.0.19 Newtonsoft.Json, 13.0.3

In [34]:
var loanNumber = await Kernel.GetInputAsync("Loan Number");
if (string.IsNullOrWhiteSpace(loanNumber))
{
  return;
}

Print("Fetching Amp Clients & MC Clients...");
var mcClientsTask = mcServiceClient.GetClientsAsync(loanNumber);
var ampClientsTask = ampServiceClient.GetClients(loanNumber);
await Task.WhenAll(mcClientsTask, ampClientsTask);

Print("✓ Completed Fetch AMP Clients & MC Clients...");
var mcClientsResult = await mcClientsTask;
mcClientsResult.EnsureSuccessStatusCode();
var ampClients = (await ampClientsTask)?.ToList();

var orderedMCClients = mcClientsResult.Result.OrderBy(c => c.IsPrimaryBorrower? 0: 1).ToList();
orderedMCClients.Display();
ampClients.Display();
Print(new string('-', 50));

VerifyClientsMatch(orderedMCClients, ampClients);

// Console.WriteLine("Checking for GCID mismatch");

// Console.WriteLine("Check of Duplicate GCID");





Fetching Amp Clients & MC Clients...

Completed Fetch AMP Clients & MC Clients...

MC Clients
Name        Client UID                        GCID      Active  
Suzi Build  b63d24803dad4cd49fbeb9d2f850665b  61501812  ✓       

Amp Clients
Name        GCID      
Suzi Build  61501812  

--------------------------------------------------

Checking if clients numbers match..

❌ MC and AMP Client Counts Match


<null>